In [6]:
# Load LSTM network and generate text
import sys
import numpy
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint


# load csv into list file
lis=[]
with open('/Users/dzy/Desktop/变量/Extracurricular/Deep Learning Music Composition/MusicComposition/data/processed_csv_bin/83.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            a=row[0].split()
            lis.append([int(i) for i in a])


# summarize the loaded data
totalmidiclicks = len(lis)
print("Total midi clicks: ", totalmidiclicks)


# prepare the dataset of input to output pairs encoded as integers
seq_length = 64
dataX = []
dataY = []
for i in range(0, totalmidiclicks - seq_length, 1):
    seq_in = lis[i:i + seq_length]
    seq_out = lis[i + seq_length]
    dataX.append([singleclick for singleclick in seq_in])
    dataY.append(seq_out)
n_patterns = len(dataX)
print("Total iterations: ", n_patterns)


# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 128))
Y = numpy.reshape(dataY,(n_patterns,128))



# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


# fit the model
model.fit(X, Y, epochs=1000, batch_size=64, callbacks=callbacks_list)

Total midi clicks:  5558
Total iterations:  5494
Epoch 1/1000
5494/5494 [==============================] - 49s 9ms/step - loss: 9.0771

Epoch 00001: loss improved from inf to 9.07709, saving model to weights-improvement-01-9.0771-bigger.hdf5
Epoch 2/1000


KeyboardInterrupt: 

In [8]:
from keras.models import model_from_json
model.load_weights("/Users/dzy/Desktop/变量/Extracurricular/Deep Learning Music Composition/MusicComposition/weights-improvement-149-3.2727-bigger.hdf5", by_name=False)
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64, 256)           394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
Total params: 952,448
Trainable params: 952,448
Non-trainable params: 0
_________________________________________________________________


In [10]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")

music = pattern

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 128))
    prediction = model.predict(x, verbose=0)
    
    result = numpy.round(prediction*35)
    result = numpy.where(result>1,1,result)
    seq_in = [value for value in pattern]
    listresult = result.tolist()
    listresult = [[int(i) for i in listresult[0]]]
    seq_in.append(listresult[0])
    pattern = seq_in[1:]
    music.append(listresult[0])
print(music)



with open('music83_150epoch_64seqlength_1000length_*35.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for n in music:
        m = str(n)
        m = m.replace(']','\n')
        m = m.replace('[','')
        m = m.replace(']','')
        m = m.replace(',','')
        csvfile.write(m)



Seed:
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [48]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print()

readytowrite = pattern
print(readytowrite)

# generate characters
for i in range(1000):
    print("dude")
    x = numpy.reshape(pattern, (1, len(pattern), 128))
    prediction = model.predict(x, verbose=0)
    
    result = numpy.round(prediction)
    seq_in = [value for value in pattern]
    pattern.append(result)
    pattern = pattern[1:len(pattern)]
    readytowrite.append(result)


Seed:

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

ValueError: cannot reshape array of size 16 into shape (1,16,128)

In [ ]:
# Load LSTM network and generate text
import sys
import numpy
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint


# load csv into list file
lis=[]
with open('whatever.csv', 'rb') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            lis.append([int(i) for i in row.split()])


# summarize the loaded data
totalmidiclicks = len(lis)
print("Total midi clicks: ", totalmidiclicks)


# prepare the dataset of input to output pairs encoded as integers
seq_length = 16
dataX = []
dataY = []
for i in range(0, totalmidiclicks - seq_length, 1):
    seq_in = lis[i:i + seq_length]
	seq_out = lis[i + seq_length]
	dataX.append([singleclick for singleclick in seq_in])
	dataY.append(seq_out)
n_patterns = len(dataX)
print("Total iterations: ", n_patterns)


# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 128))
Y = numpy.reshape(dataY,(n_patterns,128))



# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))


# load the network weights
filename = "weights-improvement-19-1.9435.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')






# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")


# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]